<a href="https://colab.research.google.com/github/lakshyasharma14/bro-code/blob/master/public/static/data/set_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Set Predictor

Imports

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import gradient_descent_v2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain, combinations
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from google.colab import files
import csv
import random

Google Authentication to access the data sheet

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
auth.authenticate_user()

In [8]:
#random number dataset
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1bRsu9oH3N5A6N7rGFPZ3pngCvwfuHw4_57V_YCHUj38/edit?usp=sharing')
sheet = wb.worksheet('Sheet1')
raw_data = sheet.get_all_values()
df = pd.DataFrame(raw_data)

Tokenizing Data

In [9]:
data=[]
for inner in raw_data:
  tmp = []
  for str in inner:
    if str != '':
      tmp.append(str)
  data.append(tmp)

tok_data = {y for x in data for y in x}

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(tok_data)

sequences =tokenizer.texts_to_sequences(data)
random.shuffle(sequences)

Preparing data for training

In [11]:
FEATURE_SET_SIZE = max(list(map(max, *sequences)))+2
FEATURE_SET_SIZE

65

In [12]:
def binarize(num):
  dinarized_list = np.zeros(FEATURE_SET_SIZE, dtype=int)
  dinarized_list[num] = 1
  return dinarized_list;

def subset_list (num):
    s = range(num)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))

def prepare_data(lst):
  result = np.zeros(FEATURE_SET_SIZE, dtype=int)
  for index in lst:
    result[index] = 1
  return result

def split_list(a_list):
    part = int(len(a_list)//3)
    return a_list[part:], a_list[:part]


In [13]:
input= []
output=[]
for row in sequences:
  row_len = len(row)
  for subset in subset_list(row_len):
    input_sum = np.zeros(FEATURE_SET_SIZE, dtype=int)
    output_sum = np.zeros(FEATURE_SET_SIZE, dtype=int)
    for index in range(row_len):
      if index in subset:
        input_sum =  [a + b for a, b in zip(input_sum, binarize(row[index]))]
      else:
        output_sum =  [a + b for a, b in zip(output_sum, binarize(row[index]))]
    input.append(input_sum)
    output.append(output_sum)

input= np.array(input)
output = np.array(output)
input

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
input, test_input= split_list(input)
output, test_output= split_list(output)

Training model

In [ ]:
model = Sequential()

#
model.add(Dense(FEATURE_SET_SIZE*2, activation='relu', input_dim=FEATURE_SET_SIZE))
model.add(Dropout(0.5))
model.add(Dense(FEATURE_SET_SIZE*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(FEATURE_SET_SIZE, activation='softmax'))

sgd= gradient_descent_v2.SGD(lr=0.001,decay=1e-6,momentum=0.9,nesterov=True)

In [17]:
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [21]:
model.fit(input,output,epochs=3,batch_size=50)

Epoch 1/3
1253/1253 [==============================] - 2s 2ms/step - loss: 0.1562 - accuracy: 0.0589
Epoch 2/3
1253/1253 [==============================] - 2s 2ms/step - loss: 0.1358 - accuracy: 0.0687
Epoch 3/3
1253/1253 [==============================] - 2s 2ms/step - loss: 0.1292 - accuracy: 0.0733


Testing model


In [22]:
score= model.evaluate(test_input,test_output,batch_size=50)

627/627 [==============================] - 1s 1ms/step - loss: 0.1137 - accuracy: 0.0673


In [20]:
score

[0.1561989039182663, 0.07134189456701279]

Time to test, with custom input


In [23]:
test_list=["j"]
test_list = tokenizer.texts_to_sequences(test_list)
tmp_input=[]
tmp_input.append(prepare_data(test_list))
tmp_input.append(prepare_data(([1])))
tmp_input= np.array(tmp_input)
result = model.predict(tmp_input,batch_size=None,verbose=0,steps=None)
indices = (-result[0]).argsort()[:]
predicted_token = [x for x in indices if x not in test_list and x!= 0][:4]
final_prediction = tokenizer.sequences_to_texts([predicted_token])
final_prediction

['f o g k']

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
files.download('model.json')

Saved model to disk


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>